In [1]:
import pandas as pd

import glob
from tqdm import tqdm
tqdm.pandas()
import os
import sys
import wget
import requests
import urllib
from bs4 import BeautifulSoup

In [2]:
data_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/back-to-basics-data'
folder = 'C:/Users/oryan/Documents/mergers-in-cosmos'
student_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/followup-data'
results_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/back-to-basics-results'
cosmos_folder = 'E:/cosmos-data'
image_folder = 'E:/cosmos-fits'

In [3]:
df = pd.read_csv(f'{data_folder}/cosmos-primary-data.csv', index_col = 0).reset_index().rename(columns = {'index' : 'SourceID'})

In [4]:
df

,SourceID,ID,ALPHA_J2000,DELTA_J2000,X_IMAGE,Y_IMAGE,ERRX2_IMAGE,ERRY2_IMAGE,ERRXY_IMAGE,FLUX_RADIUS,...,ez_ssfr_p025,ez_ssfr_p160,ez_ssfr_p500,ez_ssfr_p840,ez_ssfr_p975,ez_Av_p025,ez_Av_p160,ez_Av_p500,ez_Av_p840,ez_Av_p975
0,4000705532455,816891,150.679846,2.196543,9200.399414,21737.232422,0.000052,0.000059,-1.043282e-05,5.535906,...,-8.931825e+00,-8.911355e+00,-8.899668e+00,-8.883307e+00,-8.872990e+00,0.529107,0.560142,0.599624,0.630920,0.652330
1,4000705532984,857121,150.673667,2.226291,9348.870117,22451.160156,0.000002,0.000004,1.265998e-08,8.876858,...,-1.102382e+01,-1.082684e+01,-1.061214e+01,-1.008995e+01,-9.884142e+00,0.342150,0.435647,0.632192,0.834451,0.937351
2,4000705533312,873195,150.668102,2.242849,9482.499023,22848.505859,0.000007,0.000002,1.829277e-06,5.542504,...,-1.036331e+01,-1.018712e+01,-1.005083e+01,-9.911050e+00,-9.815660e+00,1.108623,1.258513,1.362733,1.408040,1.445569
3,4000705533383,861738,150.645118,2.237538,10033.689453,22720.843750,0.000039,0.000078,1.332813e-05,5.169795,...,-9.574272e+00,-9.548649e+00,-9.503962e+00,-9.439343e+00,-9.384205e+00,0.240042,0.281559,0.330791,0.384371,0.436491
4,4000705539529,1284864,149.686223,2.637412,33027.406250,32317.517578,0.000031,0.000030,1.244353e-06,6.412690,...,-9.669377e+00,-9.614943e+00,-9.570104e+00,-9.533121e+00,-9.476167e+00,0.094187,0.114436,0.140990,0.159430,0.183608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3781,6000535064806,1041748,150.435644,2.404769,15058.375977,26733.025391,0.000284,0.000249,1.584763e-05,3.683371,...,-8.715841e+00,-8.601144e+00,-8.466426e+00,-8.338096e+00,-8.265055e+00,0.216840,0.269234,0.351659,0.400523,0.462082
3782,6000535121007,997393,149.712229,2.359027,32405.687500,25635.718750,0.000051,0.000031,8.479790e-06,9.109251,...,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,NaN,NaN,NaN,NaN,NaN
3783,6000535781610,1231637,150.428931,2.588176,15220.276367,31134.923828,0.000496,0.000437,-8.376359e-05,3.791407,...,-9.657442e+00,-9.606500e+00,-9.499937e+00,-9.339849e+00,-9.242270e+00,0.021844,0.021844,0.062948,0.119779,0.160169
3784,6000535864481,1042682,150.505365,2.407106,13386.495117,26789.533203,0.011980,0.011985,-7.998707e-07,4.652697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df[['SourceID', 'ALPHA_J2000', 'DELTA_J2000']].rename(columns = {'SourceID' : 'object', 'ALPHA_J2000' : 'ra', 'DELTA_J2000' : 'dec'}).set_index('object')[:1000].to_csv(f'{data_folder}/primary-manifest.csv')

### Cutouts from Program Interface

In [6]:
df_red = df[['SourceID', 'ALPHA_J2000', 'DELTA_J2000']][:1000]

In [7]:
df_red

,SourceID,ALPHA_J2000,DELTA_J2000
0,4000705532455,150.679846,2.196543
1,4000705532984,150.673667,2.226291
2,4000705533312,150.668102,2.242849
3,4000705533383,150.645118,2.237538
4,4000705539529,149.686223,2.637412
...,...,...,...
995,4000880269618,150.432854,2.060154
996,4000880269747,150.410233,2.066357
997,4000880269777,150.399205,2.064778
998,4000880269790,150.411381,2.065189


In [8]:
baseurl = 'https://irsa.ipac.caltech.edu/cgi-bin/Cutouts/nph-cutouts'

In [9]:
def create_url(ra, dec, baseurl):
    
    url = f'{baseurl}?mission=COSMOS&max_size=180&locstr={ra}+{dec}&sizeX=30&units=arcsec&ntable_cutouts=18&cutouttbl1=acs_mosaic_2.0'
    
    return url

In [10]:
df_urls = df_red.assign( url = df_red.apply(lambda row: create_url(row.ALPHA_J2000, row.DELTA_J2000, baseurl), axis = 1) )

In [11]:
def download_im(id_str, link, files, folder):
    
    if f'{folder}\\{id_str}.fits' in files:
        return True
    
    result = urllib.request.urlopen(link).read()
    soup = BeautifulSoup(result)
    links = soup.find_all('a')
    link_found = False
    for i in links:
        url = i.get('href', None)
        if '.fits' in url:
            link_found = True
            break
    if not link_found:
        return False
        
    filename = wget.download(url, out=folder)
    os.rename(filename, f'{folder}/{id_str}.fits')
    
    return True

In [12]:
files = glob.glob(f'{image_folder}/*.fits')
df_downloaded = (
    df_urls.assign(
        downloaded = df_urls.progress_apply(lambda row: download_im(row.SourceID, row.url, files, image_folder), axis = 1)
    )
)

100%|██████████| 1000/1000 [3:42:30<00:00, 13.35s/it] 
